# Simple HMM in NumPy or PyTorch

In [ ]:
%run /home/hanzl/f-w/prak/acmodel/matrix.py

In [ ]:
A = m([[0.7,0.3,0,0],
       [0,0.9,0.1,0],
       [0,0,0.8,0.2],
       [0,0,0,1]])

In [ ]:
# where we can be
x = m([[1,0,0,0]])

In [ ]:
# recording length in frames
tmax = 14000

In [ ]:
#len_x = len(x.T)
len_x = x.size()[1]

#alpha_m = m.zeros((tmax,len_x)) # mantissa-like values, kept in reasonable range
alpha_m = m.rowlist((tmax,len_x)) # mantissa-like values, kept in reasonable range


#alpha_exp = m.zeros(tmax)
alpha_exp = m.rowlist((tmax,1))


exponent = 0
base = 1000 # base for separation to exp and m (somewhere around 5e-324 starts underflow)

In [ ]:
#b = alpha_m + 0.1

b = m.zeros((tmax,len_x)) + 0.1


In [ ]:
x_m = m([[1,0,0,0]]) 

In [ ]:
%%time
x_m = m([[1,0,0,0]]) # mantissa-like moderated value (kept in range)
for row in range(tmax):
    while x_m.max()<1/base: # renormalize and remember power of base used
        x_m *= base
        exponent -= 1
    alpha_exp[row] = exponent
    alpha_m[row] = x_m
    x_m = x_m@A*b[row]   # FINETUNE WHICH row's b IS USED (ALSO FOR beta)

Wall time: 6.77 s for GPU, 1.44 s for CPU (both PyTorch sparse)





# Backward

In [ ]:
A

In [ ]:
x_m

In [ ]:
#At = np.transpose(A)
At = A.T()


In [ ]:
#beta_m = m.zeros((tmax,len_x))
beta_m = m.rowlist((tmax,len_x))


#beta_exp = m.zeros(tmax)
beta_exp = m.rowlist((tmax,1))

exponent = 0

In [ ]:
x_m = m([[0,0,0,1]])
for row in range(tmax-1,0-1,-1):
    beta_m[row] = x_m
    beta_exp[row] = exponent
    x_m = x_m@At*b[row]
    
    while x_m.max()<1/base: # renormalize and remember power of base used
        x_m *= base
        exponent -= 1

## important tricky conversions in place:
Can be done just once, fix these hacks

In [ ]:
alpha_exp = m(alpha_exp.val)

In [ ]:
beta_exp = m(beta_exp.val)

In [ ]:
L_m = alpha_m*beta_m # this is .*

In [ ]:
alpha_m

In [ ]:
beta_m

In [ ]:
L_m

In [ ]:
L_exp = alpha_exp+beta_exp

In [ ]:
L_exp.val.to_dense().max()

In [ ]:

#L_exp -= L_exp.max()


#L_exp -= L_exp.val.to_dense().max()

L_exp += float(-L_exp.val.to_dense().max())



In [ ]:
# re-normalize L

In [ ]:
L_m.max()

In [ ]:


L_md = L_m.val.to_dense()
L_expd = L_exp.val.to_dense()


In [ ]:
#for row in range(tmax):
#    while L_exp[row]<0: # renormalize and remember power of base used
#        L_m[row] *= 1/base
#        L_exp[row] += 1
        
        
        
#for row in range(tmax):
#    while float(L_exp[row].val)<0: # renormalize and remember power of base used
#        L_md[row] *= 1/base
#        L_exp[row] += 1
      
        
for row in range(tmax):
    while L_expd[row]<0: # renormalize and remember power of base used
        L_md[row] *= 1/base
        L_expd[row] += 1
      
     

In [ ]:
L_expd.min(), L_expd.max()

In [ ]:
#list(L_m.sum(axis=1))

list(L_md.sum(axis=1))


In [ ]:
L_md

In [ ]:
!pwd